#Продвинутый Python, ДЗ-2

Правила игры:

В домашке 8 задач, разбаловка указана в задании. Суммарно за дз можно получить 100 баллов, что равняется 10 баллам

В каждой задаче необходимо реализовать функцию, которая после будет проверяться через github classroom на тестах. Сами тесты лежит в гитхабе, можете локально проверить работу функций перед сдачей

Дедлайн - 7 дней после выдачи дз. Необходимо залить решеннный ноутбук в github и прислать ссылку в Anytask (без выполнения любого из пунктов работа проверяться не будет)

В данной домашке нужно использовать pandas

In [12]:
# https://stackoverflow.com/a/60658965/7286121

from IPython.core.magic import register_cell_magic

@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode, encoding='utf-8') as f:
        f.write(cell)
    get_ipython().run_cell(cell)

## Данные

В этом домашнем задании вам придется оказаться на месте аналитика в бразильском маркетплейсе [Olist](https://olist.com/pt-br/). Вам необходимо исследовать данные и на их основании сделать выводы, которые помогут бизнесу расцветать!

Данные находятся тут в файле archive.zip

In [ ]:
#!pip install wget
#перед сдачей это закомментить

In [13]:
# Код, который будет в каждом тесте, названия не менять
import pandas as pd
import numpy as np
import zipfile
import wget

url = 'https://github.com/Palladain/Deep_Python/raw/main/Homeworks/Homework_1/archive.zip'
filename = wget.download(url)

with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./')

customers = pd.read_csv('olist_customers_dataset.csv')
location = pd.read_csv('olist_geolocation_dataset.csv')
items = pd.read_csv('olist_order_items_dataset.csv')
payments = pd.read_csv('olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')
translation = pd.read_csv('product_category_name_translation.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')

100% [........................................................................] 44717580 / 44717580

Ван дано 9 датасетов, которые содержат в себе все данные по 100 000 заказам со всей Бразилии. Чтобы облегчить вам жизнь, вот связи по этим датасетам (файл product_category_name_translation является переводом названий категорий с португальского на английский)

![](https://i.imgur.com/HRhd2Y0.png)

Ну что же, начнем!

## Задание 1 (10 баллов)

Определите:

* Число товаров

* Среднюю стоимость товара (стоимость товара = среднее от цен в датасете items)

в разрезе категорий (все категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
category | products | price

value    | value      | value
```

**Обратите внимание:**

Для категории portateis_cozinha_e_preparadores_de_alimentos перевод portable kitchen and food preparers

Для категории pc_gamer перевод PC Gamer

Для них нужно отдельно добавить перевод

In [111]:
%%write_and_run task_1.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.express as px
from dateutil.parser import parse
import random

category_translation = {
    'portateis_cozinha_e_preparadores_de_alimentos': 'portable kitchen and food preparers',
    'pc_gamer': 'PC Gamer'
}

def task_1(translation, items, products):
    # Translate product category names.
    translation['product_category_name'] = translation['product_category_name'].replace(category_translation)
    
    # Merging dataframes based on common columns.
    merged_data = pd.merge(items, products, on='product_id')
    merged_data = pd.merge(merged_data, translation, on='product_category_name')
    
    # Group by English category name and aggregate data.
    result = merged_data.groupby('product_category_name_english').agg({'product_id': 'count', 'price': 'mean'}).reset_index()
    result.columns = ['category', 'products', 'price']
    
    return result


In [112]:
#Проверки

res = task_1(translation, items, products)

assert res[res.category == 'portable kitchen and food preparers'].price.values[0] == 186.996
assert len(res) == 73
assert len(res.drop_duplicates()) == 73
assert res[res.category == 'drinks'].products.values[0] == 81
assert res.products.sum() == 32341
assert res.price.sum() == 12459.751444351941
assert res[res.category == 'home_confort'].price.values[0] == 185.56926417326417

IndexError: index 0 is out of bounds for axis 0 with size 0

## Задание 2 (10 баллов)

Определите для каждого продавца основную категорию их продаж (категории должны быть на английском языке)

Табличка, которая у вас должна получиться:

```
seller_id | category

value    | value
```




In [94]:
%%write_and_run task_2.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_2(translation, products, items):
    # YOUR CODE
    products_translated = products.merge(translation, on='product_category_name', how='inner')
    seller_categories = items.merge(products_translated, on='product_id', how='inner')
    mode_function = lambda x: x.value_counts().index[0] if not x.empty else np.nan
    seller_main_category = seller_categories.groupby('seller_id')['product_category_name_english'].agg(mode_function).reset_index()
    seller_main_category.rename(columns={'product_category_name_english': 'category'}, inplace=True)
    return seller_main_category
result = task_2(translation, products, items)



                             seller_id                category
0     0015a82c2db000af6aaaf3ae2ecb0532        small_appliances
1     001cca7ae9ae17fb1caed9dfb1094831            garden_tools
2     001e6ad469a905060d959994f1b41e4f          sports_leisure
3     002100f778ceb8431b7a1020ff7ab48f         furniture_decor
4     004c9cd9d87a3c30c522c48c4fc07416          bed_bath_table
...                                ...                     ...
3028  ffcfefa19b08742c5d315f2791395ee5  books_general_interest
3029  ffdd9f82b9a447f6f8d4b91554cc7dd3              housewares
3030  ffeee66ac5d5a62fe688b9d26f83f534         home_appliances
3031  fffd5413c0700ac820c7069d66d98c89              housewares
3032  ffff564a4f9085cd26170f4732393726                    auto

[3033 rows x 2 columns]


In [95]:
# Проверки

res = task_2(translation, products, items)
assert res[res.seller_id == 'e3e15e2c0b9700561efac21c6be48066'].category.values[0] == 'housewares'
assert res[res.seller_id == '2f73e04d12cdf0c945ded66bb3fcf6c7'].category.values[0] == 'garden_tools'
assert len(res) == len(res.drop_duplicates())
assert len(res) == 3035
assert len(res[res.category == 'telephony']) == 66
assert list(np.sort(res.groupby("category").agg({"seller_id": "nunique"}).seller_id.values)) == [  1,   1,   1,   1,   1,   2,   2,   2,   2,   2,   3,   3,   4,
         4,   4,   4,   5,   5,   5,   5,   5,   5,   6,   6,   6,   7,
         8,  10,  12,  13,  13,  13,  14,  14,  14,  15,  16,  17,  17,
        17,  19,  20,  20,  20,  21,  22,  26,  37,  37,  43,  46,  51,
        54,  59,  66,  66,  78,  87,  87,  99, 101, 116, 125, 156, 216,
       224, 256, 288, 310]

AssertionError: 

## Задание 3 (10 баллов)

Выведите долю потраченных денег по каждому штату (потраченные деньги - сумма денег по доставленным заказам, сумма по price и freight_value)

*Примечание:* разбивка по штатам - по штату покупателя, процент - число от 0 до 1


Табличка, которая у вас должна получиться:

```
state | perc

value | value
```

In [28]:
%%write_and_run task_3.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_3(orders, customers, items):
    # YOUR CODE
    pass

In [ ]:
# Проверки

res = task_3(orders, customers, items)
assert res.perc.sum() == 1
assert res[res.state == "RS"].perc.values[0] == 0.055868056429816286
assert res.sort_values("perc", ascending=True).iloc[0, 1] == 0.0005862290943146945
assert res.sort_values("perc", ascending=False).iloc[0, 1] == 0.3741756035817322
assert len(res) == 27

## Задание 4 (10 баллов)

Определите средний чек покупки (добавьте разбивку на стоимость самого заказ и стоимость доставки) и среднее число товаров в заказе

А также определите среднее число покупок на пользователя (обратите внимание на идентификаторы)

In [72]:
%%write_and_run task_4.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_4(items, orders, customers):
    # YOUR CODE
    orders_items = pd.merge(orders, items, on="order_id")
    avg_product_cost = orders_items.groupby('order_id')['price'].sum().mean()
    avg_delivery_cost = orders_items.groupby('order_id')['freight_value'].sum().mean()

    # Average number of items in each order
    avg_items_per_order = orders_items.groupby('order_id')['order_item_id'].count().mean()

    # Average number of purchases per unique customer
    avg_purchases_per_customer = orders['order_id'].nunique() / customers['customer_unique_id'].nunique()

    return avg_product_cost, avg_delivery_cost, avg_items_per_order, avg_purchases_per_customer
result = task_4(items, orders, customers)

print(result)

(137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0348089410589412)


In [73]:
# Проверка

res = task_4(items, orders, customers)
assert res == (137.7540763788945, 22.823561713254815, 1.1417306873695092, 1.0348089410589412)

## Задание 5 (10 баллов)

Посчитайте CSAT (customer satisfaction - средняя оценка ревью) и отобразите средний CSAT по дням в период с апреля 2017 по апрель 2018 года

Все манипуляции с датой необходимо сделать с помощью datetime и dateutil


Табличка, которая у вас должна получиться:

```
date | csat

"YYYY-MM-DD" | value
```

In [74]:
%%write_and_run task_5.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_5(reviews):
    # YOUR CODE
    reviews['date'] = pd.to_datetime(reviews['review_creation_date'])

    start_date ='2017.04.01'
    end_date = '2018.05.01'
    reviews_f = reviews[(reviews['date'] >=  start_date) & (reviews['date'] <  end_date)]
    reviews_f['date'] = reviews_f['date'].dt.strftime("%Y-%m-%d")

    csat_by_day = reviews_f.groupby(['date'], as_index = False).mean()
    csat_by_day.columns = ['date', 'csat']

    return csat_by_day

In [75]:
# Проверки
res = task_5(reviews)
assert res.date.min() == '2017-04-01'
assert res.date.max() == '2018-04-30'
assert res.csat.sum() == 1551.8881071384853
assert res[res.date == '2017-07-11'].csat.values[0] == 4.291390728476821
assert res[res.date == '2018-02-09'].csat.values[0] == 3.992156862745098
assert res[res.csat == 3.6814814814814816].date.values[0] == '2018-02-25'

C:\Users\LALALALISAAA\AppData\Local\Temp\ipykernel_9968\1983617713.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



TypeError: Could not convert 792c280ad20ea5022c042cf683c9d78fbe7e2989673cb2a147b87ba73277da9ef6c253fad685074f6572f8b407738b33b5bc37944193ed6f23aa5c130c003c5435771f30f806e6c813693c8ce39757e1ca09d5950900c3a43903e614c588e791ddca2624a2c1a8317c2360e2f352ee5eb74d391d225a284210db00929a8adb1cb8e95119ead0491849038190a8021b05f912f2d5a8ea53b18eb61722d571d9216ed8048ebe1662ac0b3538e9217a2734f7bf1007c7c27dc580b25fb3b1199d96888bf415256c6c641b44edac8fda1b52b776a4e3ebdba6845b2acb50c756a3e2e12151267e4594d69eda14a871e2a1abe8938a51136a62c4a86efa745dca7a427872e05b1046f59e3786f6afb63f13bb302058393a6ba408cb5047cdf566bc6c86f5c8876ff48350ec9a046351a7e26fa29fa7ae34e5742cddb1d43ecbcfaf5f66bddd13d697c95984edf652a7bcb19c4c9dedac236b368243f0ff31368191ac21f093e8633c9296286aca13f37a839d41d614b133efebcd10352001bc024071bb196ca8b0f72505563f4be6f7376917ddfb8b6026acfd79f54bb22e66df75bb831a433123490c097c9170779a2b3d33c950324a42c5796d06f569f77d8b2e889433496809316281ce3edab57b9382ddff62bf4860625b892e212ce1d9549527de8c69dab0f

## Задание 6 (10 баллов)

Посмотрите, как быстро отвечают пользователи (сделайте аггреграцию по числу дней ответа) и какая средняя оценка

Все манипуляции со временем нужно делать через datetime и dateutil

Табличка, которая у вас должна получиться:

```
days | csat | orders

value | value | value
```

Результаты должны быть отсориртированы по дня по возрастанию

In [84]:
%%write_and_run task_6.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd
from dateutil.parser import parse
import datetime

def task_6(reviews):
    # YOUR CODE
    reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
    reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])
    reviews['days'] = (reviews['review_answer_timestamp'] - reviews['review_creation_date']).dt.days
    result = reviews.groupby('days').agg({'review_score': 'mean', 'order_id': 'count'}).reset_index()
    result.rename(columns={'review_score': 'csat', 'order_id': 'orders'}, inplace=True)
    result = result.sort_values(by='days')

    return result
reviews = pd.read_csv('olist_order_reviews_dataset.csv')
res = task_6(reviews)
print(res)

     days      csat  orders
0       0  3.886663   24361
1       1  4.235586   30995
2       2  4.048685   15898
3       3  4.194567   14062
4       4  4.042382    4601
..    ...       ...     ...
209   446  5.000000       1
210   471  4.000000       1
211   508  5.000000       1
212   512  5.000000       1
213   518  1.000000       1

[214 rows x 3 columns]


In [85]:
# Проверки
res = task_6(reviews)
assert res.orders.sum() == 99224
assert np.all(res.days.values == np.sort(res.days.values))
assert len(res) == 214
assert res.days.min() == 0
assert res.days.max() == 518
assert res[res.days == 233].csat.values[0] == 3.0
assert res[res.days == 87].orders.values[0] == 4

## Задание 7 (10 баллов)

Выделите все заказы, где не проставлено поле order_delivered_customer_date. Замените его на дату '2999-12-31'

In [45]:
%%write_and_run task_7.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_7(orders):
    # YOUR CODE
    orders['order_delivered_customer_date'].fillna('2999-12-31', inplace=True)
    return orders


In [46]:
#Проверки

res = task_7(orders)
assert len(res[res.order_delivered_customer_date.isna()]) == 0
assert len(res) == 99441
assert len(res[res.order_delivered_customer_date == '2999-12-31']) == 2965

## Задание 8 (30 баллов)

Определите топ-10 продавцов, у которых больше 100 заказов, отсылающие чаще всего свою посылку в другие регионы (считаются только доставленные заказы)

Чаще всего отсылают = самый большой процент отправленных заказов в другой штат

Табличка, которая у вас должна получиться:

```
seller_id | share

value | value
```

In [ ]:
%%write_and_run task_8.py

# важно! все зависимости, которые используете (если добавляее новые) в этом классе надо явно продублировать в эту ячейку
import numpy as np
import pandas as pd

def task_8(orders, items, sellers, customers):
    # YOUR CODE
    pass

In [ ]:
# Проверки

res = task_8(orders, items, sellers, customers)
assert np.all(res.share.values == np.sort(res.share.values)[::-1])
assert res.share.values[0] == 0.9743589743589743
assert res.share.values[-1] == 0.9356435643564357
assert res.seller_id.values[5] == '1b4c3a6f53068f0b6944d2d005c9fc89'
assert res.seller_id.values[2] == '06a2c3af7b3aee5d69171b0e14f0ee87'
assert res.share.sum() == 9.519118744616716